# Bidirectional Encoder Representations from Transformers (BERT)

In [1]:
import torch 

In [2]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [3]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [4]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                                    return_dict=False) #Huggingface version update required the dictionary return to be False
    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')

#Advice for new parents 
source: mumsnet

In [5]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

context1 = """As your baby gets a bit older, she won't need as many feeds during the night. As such, she'll be able to sleep for a bit longer. By this stage, some babies will even sleep for a full eight hours during the night (although, word of warning: don't get your hopes up). Most babies will spend twice as long sleeping at night as they do during the day – e.g. eight hours at night and four hours during the day.
Many babies aged six months to a year will no longer need a night feed and may sleep for up to 12 hours at night. However, there're no guarantees of this, and some babies may wake up in the night for a bit (or a lot) longer.
Sometimes, your baby will nod off with seemingly no effort required from you. No rocking, no shushing, no patting. Then there are the nights where your baby will fight sleep as though their little life depends on it. What's more, once she does drop off, it won't be long before she's awake again. You know she needs to sleep, but she hasn't quite got the memo.

For nights like these, we've compiled a guide for getting your baby to sleep. Whether you give controlled crying a go, or experiment with using a dummy, there're a number of things you can do to up your chances of getting your baby to sleep."""

print(wrapper.fill(context1))

As your baby gets a bit older, she won't need as many feeds during the night. As
such, she'll be able to sleep for a bit longer. By this stage, some babies will
even sleep for a full eight hours during the night (although, word of warning:
don't get your hopes up). Most babies will spend twice as long sleeping at night
as they do during the day – e.g. eight hours at night and four hours during the
day. Many babies aged six months to a year will no longer need a night feed and
may sleep for up to 12 hours at night. However, there're no guarantees of this,
and some babies may wake up in the night for a bit (or a lot) longer. Sometimes,
your baby will nod off with seemingly no effort required from you. No rocking,
no shushing, no patting. Then there are the nights where your baby will fight
sleep as though their little life depends on it. What's more, once she does drop
off, it won't be long before she's awake again. You know she needs to sleep, but
she hasn't quite got the memo.  For nig

In [6]:
question = "how many hours do the six month old babies sleep?"

answer_question(question, context1)

Query has 309 tokens.

Answer: "12 hours at night"


In [7]:
question = "how to make my baby sleep?"

answer_question(question, context1)

Query has 305 tokens.

Answer: "whether you give controlled crying a go , or experiment with using a dummy"


In [8]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

context = """Children with Autism Spectrum Disorder are often restricted, rigid, and even obsessive in their behaviors, activities, and interests. Symptoms may include: Repetitive body movements (hand flapping, rocking, spinning); moving constantly. Obsessive attachment to unusual objects (rubber bands, keys, light switches)."""

print(wrapper.fill(context))

Children with Autism Spectrum Disorder are often restricted, rigid, and even
obsessive in their behaviors, activities, and interests. Symptoms may include:
Repetitive body movements (hand flapping, rocking, spinning); moving constantly.
Obsessive attachment to unusual objects (rubber bands, keys, light switches).


In [9]:

question = "what are the traits of children with autism?"

answer_question(question, context)

Query has 74 tokens.

Answer: "restricted , rigid , and even obsessive"
